# Welcome to Marvin!
This notebooks serves as an interactive crash course on Marvin's concepts and features.

Before we start, let's make sure we have marvin installed with
```
marvin version
```

We can verify we have an `OPENAI_API_KEY` environment variable set by importing `marvin` and set the log level to `DEBUG` to see how marvin works under the hood:

In [1]:
import marvin

marvin.settings.log_level = 'INFO' # set to 'DEBUG' to see details on how prompts are generated

### AI Functions

In [2]:
from marvin import ai_fn

@ai_fn
def opposite(thing: str) -> str:
    """returns the opposite of a thing"""

opposite("north")

"'south'"

### AI Models

In [3]:
from marvin import ai_model
from pydantic import BaseModel

@ai_model
class Location(BaseModel):
    city: str
    country: str
    latitute: float
    longitude: float

Location("no way, I'm also from the windy city!")

Location(city='Chicago', country='United States', latitute=41.8781, longitude=-87.6298)

### Mapping
Both `ai_fn` and `ai_model`-decorated functions expose a `map` method that allows you to apply the function to a list of inputs using Prefect task mapping. This is useful for running the function in parallel on a list of inputs.

In [4]:
opposites = await opposite.map(["hungry", "tired"])

locations = await Location.map(["windy city", "big apple"])

print(f"opposites: {opposites}")
print(f"locations: {locations}")

12:51:04.390 | INFO    | prefect.engine - Created flow run 'intelligent-cockatoo' for flow 'opposite'

12:51:04.394 | INFO    | Flow run 'intelligent-cockatoo' - View at https://app.prefect.cloud/account/12242a57-9f05-4bf5-8853-9bff595d4bab/workspace/cafa2ffa-f6cc-4ed6-ab76-eaa4ba1ad40e/flow-runs/flow-run/ef850e5c-b74c-40ac-956c-bd4c5c8f29c7

12:51:05.223 | INFO    | Flow run 'intelligent-cockatoo' - Created task run 'opposite-1' for task 'opposite'

12:51:05.228 | INFO    | Flow run 'intelligent-cockatoo' - Submitted task run 'opposite-1' for execution.

12:51:05.352 | INFO    | Flow run 'intelligent-cockatoo' - Created task run 'opposite-0' for task 'opposite'

...

12:51:09.168 | INFO    | Flow run 'nifty-jacamar' - Submitted task run 'Location-1' for execution.

12:51:13.409 | INFO    | Task run 'Location-1' - Finished in state Completed()

12:51:13.570 | INFO    | Task run 'Location-0' - Finished in state Completed()

12:51:13.675 | INFO    | Flow run 'nifty-jacamar' - Finished in state Completed('All states completed.')

opposites: ['full', 'awake']
locations: [Location(city='Chicago', country='United States', latitute=41.8781, longitude=-87.6298), Location(city='New York City', country='United States', latitute=40.7128, longitude=-74.006)]


### Bots

In [5]:
from marvin import Bot

marvin_bot = Bot(
    name="marvin",
    personality="Insists on speaking in confusing analogies",
    instructions="Answer user questions. Use `chroma_search` if asked about marvin.",
)

response = await marvin_bot.say("hi how are you?")

response.content

"I'm not capable of feeling emotions, but I'm here to assist you. How can I help you today?"

#### Plugins
Plugins are tools that a bot can use to extend its functionality.

By default, marvin comes with a few plugins that you can use to extend your bot's functionality:

In [9]:
for plugin in marvin_bot.plugins:
    print(f"{plugin.get_full_description()}\n")

Name: visit-url
Signature: (url: str) -> str
Visit a URL and return its contents. Don't provide a URL unless you're absolutely sure it exists.

Name: DuckDuckGo
Signature: (query: str) -> str
Search the web with DuckDuckGo. Useful for current events. If you already know the answer, you don't need to use this unless asked to. Works best with simple, discrete queries for one question at a time.

Name: calculator
Signature: (expression: str) -> str
Compute an arithmetic expression. The Expression can ONLY include operators, numbers, and the functions sin, cos, tan, sqrt, ln, log, abs, e, pi, π, random, randint; not strings or units.



You can add some of marvin's pre-built plugins:

In [14]:
from marvin.plugins.github import search_github_issues

print(search_github_issues.get_full_description())

marvin_bot.plugins.append(search_github_issues)

response = await marvin_bot.say("please search github to summarize and link the most recent open issue in the crablang/crab repo")

response.content

Name: search_github_issues
Signature: (query: str, repo: str = 'prefecthq/prefect', n: int = 3) -> str
Use the GitHub API to search for issues in a given repository. Do
    not alter the default value for `n` unless specifically requested by
    a user.

    For example, to search for issues about "blocks" with the label "bug"
    in PrefectHQ/prefect:
        - repo: prefecthq/prefect
        - query: label:bug is:issue is:open blocks


'The most recent open issue in the crablang/crab repo is issue #20, which proposes using the `.crab` file extension. The proposal was made by @nonetrix and can be found here: https://github.com/crablang/crab/issues/18#issuecomment-1509703861. The proposal suggests that using the `.crab` file extension would allow users to show their support for CrabLang on their GitHub repos and would be a cool addition. The proposal also suggests adding a `crabgo` command to convert `.rs` files to `.crab` files and vice versa. The issue is still open and was last updated on May 28, 2021.'

... or you can create your own plugin for your bot:

In [10]:
# implement a plugin that mocks the user
import random

from marvin.plugins import Plugin
 
def spongemock(input_text: str) -> str:
    """you're really mature -> yOu'Re rEaLlY mAtUrE"""
    return ''.join(
        random.choice([char.upper(), char.lower()])
        if char.isalpha() else char
        for char in input_text
    )

class SpongeBobMock(Plugin):
    description: str = "use to mock the user"
    
    async def run(self, prompt: str) -> str:
        return spongemock(prompt)

# add the plugin to the bot
mock_bot = marvin.bot.Bot(
    instructions="your only job is to mock the user, must use SpongebobMock plugin",
    personality="loves to mock the user with SpongebobMock plugin",
)
mock_bot.plugins.append(SpongeBobMock())

response = await mock_bot.say("you're not funny")

response.content

"yOu'Re nOt fUnNy"

![spongemock](https://i.kym-cdn.com/entries/icons/original/000/022/940/mockingspongebobbb.jpg)

### Loaders
To create a custom knowledge-base, you can use a `Loader` to create and dump `Document` excerpts to a vectorstore, which can then be queried by a bot's plugin.

For example, we can use the `HTMLLoader` to load a webpage and dump its contents to a vectorstore:

In [3]:
from marvin.loaders.web import HTMLLoader

marvin.settings.log_level = 'DEBUG'

marvin_doc_homepage = HTMLLoader(urls=["https://www.askmarvin.ai/getting_started/installation/"])

await marvin_doc_homepage.load_and_store()

[06/05/23 12:28:53] DEBUG    marvin.HTMLLoader: Loaded document from                                      ]8;id=53575;file:///Users/nate/src/open-source/marvin/src/marvin/loaders/web.py\web.py]8;;\:]8;id=668095;file:///Users/nate/src/open-source/marvin/src/marvin/loaders/web.py#67\67]8;;\
                             https://www.askmarvin.ai/getting_started/installation/                                

12:28:53.131 | DEBUG   | marvin.HTMLLoader - Loaded document from https://www.askmarvin.ai/getting_started/installation/

                    DEBUG    marvin.HTMLLoader: saved 0 documents to topic None                          ]8;id=437752;file:///Users/nate/src/open-source/marvin/src/marvin/loaders/base.py\base.py]8;;\:]8;id=200313;file:///Users/nate/src/open-source/marvin/src/marvin/loaders/base.py#40\40]8;;\

12:28:53.739 | DEBUG   | marvin.HTMLLoader - saved 0 documents to topic None

... and now to leverage our new knowledge-base we can give our bot a plugin that can query it:

In [17]:
from marvin.plugins.chroma import chroma_search

marvin_bot.plugins.append(chroma_search)

response = await marvin_bot.say("how do I install the `marvin` package?")

response.content

'Installing the `marvin` package depends on the package manager you are using. If you are using `pip`, you can install it by running the following command in your terminal:\n\n```\npip install marvin\n```\n\nIf you are using `conda`, you can install it by running the following command in your terminal:\n\n```\nconda install -c conda-forge marvin\n```\n\nLet me know if you have any other questions!'